In [121]:
# input

In [1]:
# features from input

robot = []

sensor = []

function = []

characteristics = []

repo = []

category = []

node = []

message = []

service = []

action = []

launch = []

robot_tag = []


query = {'robot':robot, "sensor":sensor, "function": function, 
         "characteristics": characteristics, "repo":repo, 
         "category":category, "node":node, "message": message, 
         "service":service, "action": action, "launch":launch, 
         "robot_tag":robot_tag}

# print(len(query.keys()))



# 改为文件输入，这样可以记录每一个task的输入，方便实验复现

In [2]:
# allow input with json file
import json
def load_query_json(path):
    with open(path, encoding="utf-8") as fp:
        query = json.load(fp)

    # lower all item in query
    for key,value in query.items():
        query[key] = [v.lower() for v in value]
    return query


query = load_query_json("./test_input.json")
print(query)

{'robot': ['turtlebot2'], 'sensor': ['rplidar'], 'function': ['create maps'], 'characteristics': ['gmapping', 'gazebo simulator'], 'repo': [], 'category': [], 'node': [], 'message': [], 'service': [], 'action': [], 'launch': [], 'robot_tag': []}


In [3]:
import os
import csv
from py2neo import Graph, Node, Relationship
# g = Graph("http://localhost:7474", user="neo4j", password="/")


g = Graph("bolt://localhost:7687", user="neo4j", password="/")
# g=Graph("bolt://localhost:7687",auth=("neo4j","/"))
from py2neo import RelationshipMatcher, NodeMatcher
relation_matcher = RelationshipMatcher(g)
node_matcher = NodeMatcher(g)

In [4]:
# 先遍历所有package，获取以package为节点辐射出去的所有边。

# print('***查点***') 
# data = graph.run('MATCH (p:Man) return p').data() 
# print(data) 


# 测试NodeMatcher 
# select_nodes=node_matcher.match('Package')
# print(len(list(select_nodes)))

data = g.run('MATCH (n:Package)-[r]-(d) RETURN n,r,d').data() 
print(len(data))

53377


In [5]:
print(data[0])

{'n': Node('Package', last_commit_date='2021/6/22', package_description='TurtleBot3 AutoRace ROS package that controls Raspberry Pi Camera, and process the image', package_name='turtlebot3_autorace_traffic_light_camera'), 'r': has_function(Node('Package', last_commit_date='2021/6/22', package_description='TurtleBot3 AutoRace ROS package that controls Raspberry Pi Camera, and process the image', package_name='turtlebot3_autorace_traffic_light_camera'), Node('Function', embeddings='-0.002790342317894101 0.14802764356136322 0.016497664153575897 -0.1561202108860016 0.1774379462003708 0.3015390932559967 -0.052227187901735306 0.49079298973083496 -0.2684989869594574 -0.6767650246620178 -0.3635019361972809 0.163320392370224 0.05654335767030716 0.3747364282608032 -0.4832547605037689 0.07144875079393387 0.0040234834887087345 0.27733808755874634 -0.06025496870279312 0.18899403512477875 -0.13154467940330505 -0.14824332296848297 -0.5628939867019653 -0.18358534574508667 0.11554180085659027 -0.338486

In [6]:
# print(data[1])
# print(data[1]['d'].labels)
# print(data[1]['n'].get('package_name'))

In [7]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
from bert_serving.client import BertClient
from tqdm import tqdm
import time

In [8]:
package_all = {}

for item in data:
    package_name = item['n'].get('package_name')
    if package_name not in package_all.keys():
        package_all[package_name] = {'robot':[], "sensor":[], "function": [], 
         "characteristics": [], "repo":[], "node":[], 
         "message": [], "service":[], "action": [], 
         "launch":[], "robot_tag":[], "category":[]}
    d = item['d']
    label = str(d.labels)
    if label == ":Category":
        package_all[package_name]["category"].append(d.get("name"))
    elif label == ":Robot":
        package_all[package_name]["robot"].append(d.get("name"))
    elif label == ":Sensor":
        package_all[package_name]["sensor"].append(d.get("name"))
    elif label == ":Characteristics": # can modify later: reshape it to numpy array early
        
        embeddings = np.fromstring(d.get("embeddings"), sep=' ').reshape((1,768))
        package_all[package_name]["characteristics"].append(embeddings)
        
    elif label == ":Function":
        embeddings = np.fromstring(d.get("embeddings"), sep=' ').reshape((1,768))
        package_all[package_name]["function"].append(embeddings)
        
    elif label == ":Repository":
        package_all[package_name]["repo"].append(d.get("name"))   
    elif label == ":Node":
        package_all[package_name]["node"].append(d.get("name"))
    elif label == ":Messsage":
        package_all[package_name]["message"].append(d.get("name")) 
    elif label == ":Service":
        package_all[package_name]["service"].append(d.get("name")) 
    elif label == ":Action":
        package_all[package_name]["action"].append(d.get("name")) 
    elif label == ":Launch":
        package_all[package_name]["launch"].append(d.get("name")) 
    elif label == ":Tag":  # 暂时先不用tag
        package_all[package_name]["robot_tag"].append(d.get("name")) 
    else:
        a = 0
        
    

In [9]:
# 计算相似度



def ld_distance(q_words, p_words):
    if len(q_words)==0 or len(p_words)==0:
        return 0
    scores = []
    for p in p_words:
        score = fuzz.partial_ratio(p, q_words[0]) # assume just one word in q-word
        scores.append(score)
    return max(scores)/100


def sgn_similarity(q_words, p_words):
    if len(q_words)==0 or len(p_words)==0:
        return 0
    # remove extension for names
    q_names = [name.split('.')[0].lower() for name in q_words]
    p_names = [name.split('.')[0].lower() for name in p_words]
    sgn = 0
    for name in q_names:
        if name in p_names:
            sgn += 1
    return sgn/len(q_words)

def cos_similar(sen_a_vec, sen_b_vec):
    vector_a = np.mat(sen_a_vec)
    vector_b = np.mat(sen_b_vec)
    num = float(vector_a*vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num/denom
    return cos


def cosine_similarity(q_words, p_words):
    if len(q_words)==0 or len(p_words)==0:
        return 0
    
    q_embeddings = q_words

    p_embeddings = p_words # [np.fromstring(word, sep=' ').reshape((1,768)) for word in p_words]

    score = 0
    for q_e in q_embeddings:
        max_cos = 0
        for p_e in p_embeddings:
            cos = cos_similar(q_e, p_e)
            if cos>max_cos:
                max_cos = cos
        score += max_cos
    return score/len(q_words)


def sim_q_p(query, p_features):
    score = {"robot":0, "sensor":0, "function": 0, 
         "characteristics": 0, "repo":0, "node":0, 
         "message": 0, "service":0, "action": 0, 
         "launch":0, "category":0}
    
    for dim in score.keys():
        if dim in ["robot", "sensor"]:
            # fuzzuwuzzy LD distance
            score[dim] = ld_distance(query[dim], p_features[dim])
        elif dim in ["function", "characteristics"]:
            # set weight: hardware match more important, set the weight to 1.0; lower the func and character to 0.8
            score[dim] = 0.8*cosine_similarity(query[dim], p_features[dim])
        elif dim in ["repo", "node", "message", "service", "action", "launch", "category"]:
            score[dim] = sgn_similarity(query[dim], p_features[dim])
        else:
            print("unknown dimension:", dim)

    return score
    
def pkg_scores(query, package_all):
    scores = {}
    scores_detail = {}
    for p_name, features in tqdm(package_all.items()):

        score = sim_q_p(query, features)
        sum_score = sum(score.values())
    
        scores[p_name] = sum_score/len(score)
        scores_detail[p_name] = score
    return scores, scores_detail



In [10]:
# test captilize symbol influence fuzzywyzzy match score
fuzz.partial_ratio("Dynamixel", "dynamixel")

89

In [11]:
from joblib import Parallel, delayed
import time, math
import multiprocessing


def pkg_score_parallel_multi_query(p_name, features):
    q_len = len(q_list)
    for i in range(q_len):
        score = sim_q_p(q_list[i], features)
        sum_score = sum(score.values())

        scores[i][p_name] = sum_score/len(score)
        scores_detail[i][p_name] = score

    
def preprocess_query_list(path):
    csv_file = open(path, 'r')
    reader = csv.DictReader(csv_file)
    bc = BertClient(ip='localhost', check_version=False, check_length=False)
    
   
    query_list = []
    for row in reader:
        # each row represent a query
        
        raw_query = row.copy()
        for dim in raw_query.keys():
            if raw_query[dim] != "":
                if dim in ['function', 'characteristics']:
                    function_wds = raw_query[dim].split(',')
                    function_embeddings = [bc.encode([word]) for word in function_wds]
                    raw_query[dim] = function_embeddings
                elif dim in ["robot", "sensor", "repo", "node", "message", "service", "action", "launch", "category"]:
                    raw_query[dim] = raw_query[dim].lower().split(',')
                else:
                    pass
            else:
                raw_query[dim] = []
                
        
        query_list.append(raw_query)
    bc.close()
    return query_list
    

In [100]:
# test velodyne repo!!!
# q_list = preprocess_query_list("./test_query.csv")
q_list = preprocess_query_list("./user_queries.csv")
q_list = q_list[92:95]
print(len(q_list))

scores = [{} for i in range(len(q_list))]
scores_detail = [{} for i in range(len(q_list))]

start = time.time()
num_cores = multiprocessing.cpu_count()
# n_jobs is the number of parallel jobs
Parallel(n_jobs=num_cores, require='sharedmem')(delayed(pkg_score_parallel_multi_query)(p_name, features) for p_name, features in package_all.items())

end = time.time()
print('{:.4f} s'.format(end-start))

sorted_scores = [[] for i in range(len(q_list))]

for i in range(len(q_list)):
#     print("user query ", i)
    sorted_scores[i] = sorted(scores[i].items(), key=lambda x: x[1], reverse=True)
    k = 1
    found = False
    for package_name, score in sorted_scores[i]:
#         print(package_name, score)
        if package_name == q_list[i]['desired_package']:
            print(k)
            found = True
            break
        k += 1
        if k>20:
            break
    if not found:
        print("NA")

3
2.3182 s
NA
17
8


In [99]:
# print(q_list[4])
id = 0
desired = "object_recognition_ros"
cur_top = "bwi_rqt_plugins"
print(scores_detail[id][desired]) # func: Includes a specialized viewer
print(scores[id][desired])
print(scores_detail[id][cur_top]) # func: depth image
print(scores[id][cur_top])

# print(package_all["velodyne_simulator"])



{'robot': 0, 'sensor': 0, 'function': 0.6104788851298708, 'characteristics': 0.0, 'repo': 0, 'node': 0, 'message': 0, 'service': 0, 'action': 0, 'launch': 0, 'category': 0}
0.05549808046635189
{'robot': 0, 'sensor': 0, 'function': 0.6757443800746835, 'characteristics': 0.0, 'repo': 0, 'node': 0, 'message': 0, 'service': 0, 'action': 0, 'launch': 0, 'category': 0}
0.061431307279516685


In [98]:
sorted_scores[0][:20]
# fuzz.partial_ratio("schunk svh", "velodyne hdl-64e 3d lidar")

# fuzz.partial_ratio("velodyne hdl-64e 3d lidar", "velodyne hdl-64e 3d lidar")

[('asr_next_best_view', 0.06833993728159865),
 ('object_recognition_capture', 0.06740028807476629),
 ('grasping_msgs', 0.06596964790023901),
 ('marker_rviz_plugin', 0.06503724114938049),
 ('class_loader', 0.06500849752928367),
 ('asr_object_database', 0.06456787634187866),
 ('asr_psm', 0.06436866306726857),
 ('osg_markers', 0.06408701590822874),
 ('turtlebot_arm_block_manipulation', 0.06387565713672536),
 ('asr_intermediate_object_generator', 0.06304491074021548),
 ('librealsense2', 0.06269249829095358),
 ('nav_core', 0.06236514606007291),
 ('rocon_hub_client', 0.062203098579749504),
 ('rqt_publisher', 0.062017992754310594),
 ('marshmallow', 0.06164576915964879),
 ('rtt_rospack', 0.061557828319618715),
 ('ez_interactive_marker', 0.06154876389555974),
 ('webtest', 0.06153591050319174),
 ('image_geometry', 0.061458645870268745),
 ('bwi_rqt_plugins', 0.061431307279516685)]

In [68]:


def test_cosine_similarity(q_words, p_words):
    if len(q_words)==0 or len(p_words)==0:
        return 0
    
    q_embeddings = q_words

    p_embeddings = p_words # [np.fromstring(word, sep=' ').reshape((1,768)) for word in p_words]

    score = 0
    for q_e in q_embeddings:
        max_cos = 0
        for p_e in p_embeddings:
            cos = cos_similar(q_e, p_e)
            if cos>max_cos:
                max_cos = cos
        score += max_cos
        print(max_cos)
    return score/len(q_words)

csim_score = test_cosine_similarity(q_list[id]["characteristics"], package_all["rqt_action"]["characteristics"])
print(csim_score)
print("xxxxxxxxxxxxxxxxx")
csim_score = test_cosine_similarity(q_list[id]["characteristics"], package_all["rqt_gui"]["characteristics"])
print(csim_score)

0.7528108781024898
1.0000000270093716
0.8764054525559307
xxxxxxxxxxxxxxxxx
0.7303898790148273
1.0000000270093716
0.8651949530120995


In [13]:
# q_list = preprocess_query_list("./test_query.csv")
q_list = preprocess_query_list("./user_queries.csv")
print(len(q_list))

scores = [{} for i in range(len(q_list))]
scores_detail = [{} for i in range(len(q_list))]

start = time.time()
num_cores = multiprocessing.cpu_count()
# n_jobs is the number of parallel jobs
Parallel(n_jobs=num_cores, require='sharedmem')(delayed(pkg_score_parallel_multi_query)(p_name, features) for p_name, features in package_all.items())

end = time.time()
print('{:.4f} s'.format(end-start))

sorted_scores = [[] for i in range(len(q_list))]

for i in range(len(q_list)):
#     print("user query ", i)
    sorted_scores[i] = sorted(scores[i].items(), key=lambda x: x[1], reverse=True)
    k = 1
    found = False
    for package_name, score in sorted_scores[i]:
#         print(package_name, score)
        if package_name == q_list[i]['desired_package']:
            print(k)
            found = True
            break
        k += 1
        if k>20:
            break
    if not found:
        print("NA")

100
101.7742 s
1
1
1
1
NA
NA
NA
NA
NA
NA
1
1
11
3
9
1
3
1
1
1
1
2
5
1
1
NA
NA
1
NA
NA
NA
NA
NA
NA
NA
NA
NA
1
1
1
1
1
1
1
1
1
1
5
1
1
1
2
NA
1
NA
NA
NA
NA
NA
NA
3
NA
NA
4
NA
NA
NA
NA
6
NA
NA
13
NA
NA
NA
4
NA
NA
NA
NA
1
1
NA
1
NA
NA
7
3
5
1
2
1
NA
NA
NA
NA
NA
8
NA
NA


In [38]:

print(scores_detail[5]["velodyne_simulator"]) # func: Includes a specialized viewer
print(scores[5]["velodyne_simulator"])
print(scores_detail[5]["schunk_libm5api"]) # func: depth image
print(scores[5]["schunk_libm5api"])

print(sorted_scores[6][:1])

print(scores_detail[6]["velodyne_description"])
sgn_similarity(q_list[6]["category"], package_all["velodyne_description"]["category"])
print(q_list[6]["category"])
print(package_all["velodyne_description"]["category"])

{'robot': 0, 'sensor': 1.0, 'function': 0.6737690876827577, 'characteristics': 0.0, 'repo': 0, 'node': 0, 'message': 0, 'service': 0, 'action': 0, 'launch': 0, 'category': 0.0}
0.15216082615297796
{'robot': 0, 'sensor': 0, 'function': 0.6531334933569282, 'characteristics': 0.0, 'repo': 0, 'node': 0, 'message': 0, 'service': 0, 'action': 0, 'launch': 0, 'category': 0.0}
0.0593757721233571
[('velodyne_pointcloud', 0.09090909090909091)]
{'robot': 0, 'sensor': 1.0, 'function': 0.0, 'characteristics': 0.0, 'repo': 0, 'node': 0, 'message': 0, 'service': 0, 'action': 0, 'launch': 0, 'category': 0.0}
description_package
['description_package']


In [ ]:
# check why robot match score = 0.0


In [29]:
print(scores_detail[4]["image_view"]) # func: Includes a specialized viewer
print(scores[4]["image_view"])
print(scores_detail[4]["laserscan_kinect"]) # func: depth image
print(scores[4]["laserscan_kinect"])
# query: func: view image

{'robot': 0, 'sensor': 0, 'function': 0.5586788402276687, 'characteristics': 0.6888185615177194, 'repo': 0, 'node': 0, 'message': 0, 'service': 0, 'action': 0, 'launch': 0, 'category': 0}
0.11340885470412619
{'robot': 0, 'sensor': 0, 'function': 0.6757501992268953, 'characteristics': 0.6496726458625729, 'repo': 0, 'node': 0, 'message': 0, 'service': 0, 'action': 0, 'launch': 0, 'category': 0}
0.12049298591722439


In [12]:
q_list = preprocess_query_list("./test_query.csv")
print(len(q_list))

scores = [{} for i in range(len(q_list))]
scores_detail = [{} for i in range(len(q_list))]

start = time.time()
num_cores = multiprocessing.cpu_count()
# n_jobs is the number of parallel jobs
Parallel(n_jobs=num_cores, require='sharedmem')(delayed(pkg_score_parallel_multi_query)(p_name, features) for p_name, features in package_all.items())

end = time.time()
print('{:.4f} s'.format(end-start))

for i in range(len(q_list)):
    print("user query ", i)
    sorted_scores = sorted(scores[i].items(), key=lambda x: x[1], reverse=True)
    k = 0
    for package_name, score in sorted_scores:
        print(package_name, score)
        k += 1
        if k==10:
            break

15
1052.3217 s
user query  0
master_discovery_fkie 0.06463298879504178
node_manager_fkie 0.06350173117424095
scenario_test_tools 0.06273337380373666
rqt_py_common 0.06263833576102835
rbcar_sim 0.06221063929985727
pmb2_simulation 0.061832918829688
std_capabilities 0.06180685656446397
fuse_msgs 0.061653303701586344
turtlebot3_example 0.0616495842489831
turtlebot_rviz_launchers 0.06163667103410497
user query  1
master_sync_fkie 0.07272727533635018
sr_robot_lib 0.06318954744540711
feed_the_troll 0.06269221805565522
qb_device_msgs 0.06242238246505629
shadow_robot 0.062374958433398416
uvc_camera 0.062283489933927336
rosjson 0.062202877479469086
qb_move_control 0.062020614496924094
qb_hand_control 0.062020614496924094
dwb_local_planner 0.06195311309518056
user query  2
rqt_robot_steering 0.06507697818906176
rqt_publisher 0.0647092371911198
rqt_topic 0.0633252521639486
rqt_service_caller 0.06299327655474331
theora_image_transport 0.06287697370982494
compressed_depth_image_transport 0.062780726

In [161]:
query = load_query_json("./query-input-3.json")
print(query)

scores = {}
scores_detail = {}

start = time.time()
num_cores = multiprocessing.cpu_count()
# n_jobs is the number of parallel jobs
Parallel(n_jobs=num_cores, require='sharedmem')(delayed(pkg_score_parallel)(p_name, features) for p_name, features in package_all.items())

end = time.time()
print('{:.4f} s'.format(end-start))

sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
k = 0
for package_name, score in sorted_scores:
    print(package_name, score)
    k += 1
    if k==10:
        break


{'robot': ['turtlebot2'], 'sensor': ['rplidar'], 'function': ['create maps'], 'characteristics': ['navigation'], 'repo': [], 'category': [], 'node': [], 'message': [], 'service': [], 'action': [], 'launch': ['move_base.launch'], 'robot_tag': []}
605.7528 s
turtlebot3_navigation 0.3343453456071001
rb1_base_navigation 0.2514691212099969
turtlebot_arm_block_manipulation 0.24065237240513945
turtlebot_description 0.2371579820248037
turtlebot_arm_object_manipulation 0.23321307817701634
turtlebot_rviz_launchers 0.2322636717256623
turtlebot_arm_description 0.23177807934775804
turtlebot_interactive_markers 0.2302121955921172
turtlebot_arm_bringup 0.2298013384189438
turtlebot3_autorace_tunnel_camera 0.22947368449927863


In [29]:
# ubuntu version 6.0 (adjust n_jobs to 4, to speed up)

query = load_query_json("./query-input-6.json")
print(query)

scores = {}
scores_detail = {}

start = time.time()
num_cores = multiprocessing.cpu_count() # 16
# n_jobs is the number of parallel jobs
Parallel(n_jobs=4, require='sharedmem')(delayed(pkg_score_parallel)(p_name, features) for p_name, features in package_all.items())

end = time.time()
print('{:.4f} s'.format(end-start))

sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
k = 0
for package_name, score in sorted_scores:
    print(package_name, score)
    k += 1
    if k==10:
        break

{'robot': ['turtlebot2'], 'sensor': [], 'function': ['use joysticks'], 'characteristics': ['teleoperate turtlebot2'], 'repo': [], 'category': [], 'node': [], 'message': [], 'service': [], 'action': [], 'launch': [], 'robot_tag': []}
595.8762 s
turtlebot_teleop 0.21628365808688763
turtlebot3_teleop 0.19812632131065736
turtlebot_bringup 0.19569661395847512
turtlebot3_autorace_camera 0.1956030374181722
turtlebot_actions 0.19467840477628975
turtlebot 0.1941377071777082
turtlebot3_example 0.1931133085502549
turtlebot_arm_block_manipulation 0.19309772891135824
turtlebot3_autorace_traffic_light_camera 0.1927630030611023
turtlebot3_autorace_parking_camera 0.1927630030611023


In [28]:
# ubuntu version 5.0 (without n_jobs, to speed up)

query = load_query_json("./query-input-5.json")
print(query)
scores, scores_detail = pkg_scores(query, package_all)
# sort scores by score
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
k = 0
for package_name, score in sorted_scores:
    print(package_name, score)
    k += 1
    if k==10:
        break

{'robot': ['turtlebot2'], 'sensor': [], 'function': ['teleoperate turtlebot2'], 'characteristics': ['use joysticks'], 'repo': [], 'category': [], 'node': [], 'message': [], 'service': [], 'action': [], 'launch': [], 'robot_tag': []}


100%|███████████████████████████████████████| 4113/4113 [16:07<00:00,  4.25it/s]

turtlebot_arm_moveit_config 0.20052876994212776
turtlebot_arm_object_manipulation 0.19587365080999586
turtlebot_bringup 0.19439625678017505
turtlebot_arm_block_manipulation 0.1942012483963131
turtlebot_rviz_launchers 0.1923628372005919
turtlebot_interactive_markers 0.19022298266645868
turtlebot 0.18980278719838228
turtlebot_stdr 0.1896949036613165
turtlebot_stage 0.1896949036613165
turtlebot3_autorace_parking_control 0.1894614193667171


In [27]:
# ubuntu version 4.0 (adjust n_jobs to 2, to speed up)

query = load_query_json("./query-input-4.json")
print(query)

scores = {}
scores_detail = {}

start = time.time()
num_cores = multiprocessing.cpu_count() # 16
# n_jobs is the number of parallel jobs
Parallel(n_jobs=2, require='sharedmem')(delayed(pkg_score_parallel)(p_name, features) for p_name, features in package_all.items())

end = time.time()
print('{:.4f} s'.format(end-start))

sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
k = 0
for package_name, score in sorted_scores:
    print(package_name, score)
    k += 1
    if k==10:
        break

{'robot': ['turtlebot2'], 'sensor': [], 'function': ['visualize turtlebot2'], 'characteristics': ['rviz'], 'repo': [], 'category': [], 'node': [], 'message': [], 'service': [], 'action': [], 'launch': [], 'robot_tag': []}
656.8734 s
turtlebot_rviz_launchers 0.2297806808472037
turtlebot_arm_moveit_config 0.2266600072726743
turtlebot_bringup 0.21335888126591243
turtlebot3_fake 0.2120978579311278
turtlebot_actions 0.20892800364371514
turtlebot_arm_object_manipulation 0.20637715643013851
turtlebot3_bringup 0.20632859914732668
turtlebot_arm_block_manipulation 0.20629082027140094
turtlebot3_example 0.20586488882975748
turtlebot 0.20545520425308955


In [26]:
# ubuntu version 3.0 (adjust n_jobs to 8, to speed up)

query = load_query_json("./query-input-3.json")
print(query)

scores = {}
scores_detail = {}

start = time.time()
num_cores = multiprocessing.cpu_count() # 16
# n_jobs is the number of parallel jobs
Parallel(n_jobs=8, require='sharedmem')(delayed(pkg_score_parallel)(p_name, features) for p_name, features in package_all.items())

end = time.time()
print('{:.4f} s'.format(end-start))

sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
k = 0
for package_name, score in sorted_scores:
    print(package_name, score)
    k += 1
    if k==10:
        break

{'robot': ['turtlebot2'], 'sensor': ['rplidar'], 'function': ['create maps'], 'characteristics': ['navigation'], 'repo': [], 'category': [], 'node': [], 'message': [], 'service': [], 'action': [], 'launch': ['move_base.launch'], 'robot_tag': []}
596.7852 s
turtlebot3_navigation 0.3020217315546337
turtlebot_navigation 0.2545454578199848
fetch_navigation 0.2530590909194813
turtlebot_rviz_launchers 0.222261987178112
rb1_base_navigation 0.2193571197538409
xbot_navi 0.21519388448309432
turtlebot_arm_block_manipulation 0.21070372611364288
turtlebot_description 0.20790820146978295
turtlebot_bringup 0.20665234207346675
turtlebot_arm_moveit_config 0.20616681750486707


In [25]:
# ubuntu version 2.0 (adjust weight 0.8)

query = load_query_json("./query-input-3.json")
print(query)

scores = {}
scores_detail = {}

start = time.time()
num_cores = multiprocessing.cpu_count()
# n_jobs is the number of parallel jobs
Parallel(n_jobs=num_cores, require='sharedmem')(delayed(pkg_score_parallel)(p_name, features) for p_name, features in package_all.items())

end = time.time()
print('{:.4f} s'.format(end-start))

sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
k = 0
for package_name, score in sorted_scores:
    print(package_name, score)
    k += 1
    if k==10:
        break

{'robot': ['turtlebot2'], 'sensor': ['rplidar'], 'function': ['create maps'], 'characteristics': ['navigation'], 'repo': [], 'category': [], 'node': [], 'message': [], 'service': [], 'action': [], 'launch': ['move_base.launch'], 'robot_tag': []}
598.2337 s
turtlebot3_navigation 0.3020217315546337
turtlebot_navigation 0.2545454578199848
fetch_navigation 0.2530590909194813
turtlebot_rviz_launchers 0.222261987178112
rb1_base_navigation 0.2193571197538409
xbot_navi 0.21519388448309432
turtlebot_arm_block_manipulation 0.21070372611364288
turtlebot_description 0.20790820146978295
turtlebot_bringup 0.20665234207346675
turtlebot_arm_moveit_config 0.20616681750486707


In [12]:
# ubuntu

query = load_query_json("./query-input-3.json")
print(query)

scores = {}
scores_detail = {}

start = time.time()
num_cores = multiprocessing.cpu_count()
# n_jobs is the number of parallel jobs
Parallel(n_jobs=num_cores, require='sharedmem')(delayed(pkg_score_parallel)(p_name, features) for p_name, features in package_all.items())

end = time.time()
print('{:.4f} s'.format(end-start))

sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
k = 0
for package_name, score in sorted_scores:
    print(package_name, score)
    k += 1
    if k==10:
        break




{'robot': ['turtlebot2'], 'sensor': ['rplidar'], 'function': ['create maps'], 'characteristics': ['navigation'], 'repo': [], 'category': [], 'node': [], 'message': [], 'service': [], 'action': [], 'launch': ['move_base.launch'], 'robot_tag': []}
598.0171 s
turtlebot3_navigation 0.33434534626147394
fetch_navigation 0.28450568183116987
turtlebot_navigation 0.27272727682043557
turtlebot_rviz_launchers 0.25510021124536725
rb1_base_navigation 0.2514691269650284
turtlebot_arm_block_manipulation 0.24065238491478083
turtlebot_description 0.23715797910995595
turtlebot_bringup 0.23558815486456067
turtlebot_arm_moveit_config 0.23498124915381113
turtlebot_arm_object_manipulation 0.23321309276149502


In [8]:
# macbook pro
query = load_query_json("./query-input-3.json")
print(query)

scores = {}
scores_detail = {}

start = time.time()
num_cores = multiprocessing.cpu_count()
# n_jobs is the number of parallel jobs
Parallel(n_jobs=num_cores, require='sharedmem')(delayed(pkg_score_parallel)(p_name, features) for p_name, features in package_all.items())

end = time.time()
print('{:.4f} s'.format(end-start))

sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
k = 0
for package_name, score in sorted_scores:
    print(package_name, score)
    k += 1
    if k==10:
        break

{'robot': ['turtlebot2'], 'sensor': ['rplidar'], 'function': ['create maps'], 'characteristics': ['navigation'], 'repo': [], 'category': [], 'node': [], 'message': [], 'service': [], 'action': [], 'launch': ['move_base.launch'], 'robot_tag': []}
605.2402 s
turtlebot3_navigation 0.3343453456071001
fetch_navigation 0.28450567899418755
turtlebot_navigation 0.2727272752202391
turtlebot_rviz_launchers 0.25510021155065016
rb1_base_navigation 0.2514691212099969
turtlebot_arm_block_manipulation 0.24065237240513945
turtlebot_description 0.2371579820248037
turtlebot_bringup 0.23558815526628676
turtlebot_arm_moveit_config 0.23498123981618527
turtlebot_arm_object_manipulation 0.23321307817701634


In [22]:
print(num_cores)
print(scores["turtlebot_navigation"])
print(scores_detail["fetch_navigation"])
print(scores_detail["turtlebot_navigation"])

#  launch 匹配有问题，去检查！！！done!!!

16
0.27272727682043557
{'robot': 0.4, 'sensor': 0, 'function': 0.7295624551180773, 'characteristics': 1.0000000450247908, 'repo': 0, 'node': 0, 'message': 0, 'service': 0, 'action': 0, 'launch': 1.0, 'category': 0}
{'robot': 1.0, 'sensor': 0, 'function': 0, 'characteristics': 1.0000000450247908, 'repo': 0, 'node': 0, 'message': 0, 'service': 0, 'action': 0, 'launch': 1.0, 'category': 0}


In [11]:
# ubuntu
query = load_query_json("./query-input-3.json")
print(query)
scores, scores_detail = pkg_scores(query, package_all)
# sort scores by score
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
k = 0
for package_name, score in sorted_scores:
    print(package_name, score)
    k += 1
    if k==10:
        break

{'robot': ['turtlebot2'], 'sensor': ['rplidar'], 'function': ['create maps'], 'characteristics': ['navigation'], 'repo': [], 'category': [], 'node': [], 'message': [], 'service': [], 'action': [], 'launch': ['move_base.launch'], 'robot_tag': []}


 14%|█████▋                                  | 589/4113 [02:14<13:25,  4.37it/s]


KeyboardInterrupt: 

In [158]:
query = load_query_json("./query-input-2.json")
print(query)
scores, scores_detail = pkg_scores(query, package_all)
# sort scores by score
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
k = 0
for package_name, score in sorted_scores:
    print(package_name, score)
    k += 1
    if k==10:
        break

{'robot': [], 'sensor': ['rplidar'], 'function': [], 'characteristics': ['gmapping', 'a2'], 'repo': [], 'category': [], 'node': [], 'message': [], 'service': [], 'action': [], 'launch': [], 'robot_tag': []}


100%|█████████████████████████████████████| 4113/4113 [16:13<00:00,  4.23it/s]

rplidar_ros 0.15618704318349447
livox_ros_driver 0.13454328346368574
velodyne_driver 0.13135381904256535
velodyne_simulator 0.13090332167850166
velodyne_pointcloud 0.13030106687525603
velodyne_laserscan 0.1295620245575738
velodyne_description 0.12922353978564463
xaxxon_openlidar 0.12824756604374668
velodyne_msgs 0.12646539688529887
velodyne_gazebo_plugin 0.12332896075750895


In [ ]:
query = load_query_json("./query-input-1.json")
print(query)
scores, scores_detail = pkg_scores(query, package_all)
# sort scores by score
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
k = 0
for package_name, score in sorted_scores:
    print(package_name, score)
    k += 1
    if k==10:
        break

In [152]:
query = load_query_json("./query-input-1.json")
print(query)
scores, scores_detail = pkg_scores(query, package_all)
# sort scores by score
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)

{'robot': ['turtlebot2'], 'sensor': [], 'function': [], 'characteristics': ['gazebo simulator'], 'repo': [], 'category': [], 'node': [], 'message': [], 'service': [], 'action': [], 'launch': [], 'robot_tag': []}


100%|█████████████████████████████████████| 4113/4113 [12:06<00:00,  5.66it/s]


In [153]:
k = 0
for package_name, score in sorted_scores:
    print(package_name, score)
    k += 1
    if k==10:
        break

turtlebot_gazebo 0.17434013879888413
turtlebot3_gazebo 0.16524922970797504
turtlebot_interactive_markers 0.15948023560693417
turtlebot_loadout_kha1 0.1587930501385091
turtlebot_description 0.1587930501385091
turtlebot_capabilities 0.1587930501385091
turtlebot 0.1587930501385091
turtlebot_arm_kinect_calibration 0.1587930501385091
turtlebot_apps 0.15689282059400822
turtlebot_create_desktop 0.15626663287944353


In [155]:
print(scores_detail["turtlebot_gazebo"])
print(scores_detail["turtlebot3_gazebo"])

{'robot': 1.0, 'sensor': 0, 'function': 0, 'characteristics': 0.9177415267877252, 'repo': 0, 'node': 0, 'message': 0, 'service': 0, 'action': 0, 'launch': 0, 'category': 0}
{'robot': 0.9, 'sensor': 0, 'function': 0, 'characteristics': 0.9177415267877252, 'repo': 0, 'node': 0, 'message': 0, 'service': 0, 'action': 0, 'launch': 0, 'category': 0}


In [138]:
k = 0
for package_name, score in sorted_scores:
    print(package_name, score)
    k += 1
    if k==10:
        break

turtlebot_arm_block_manipulation 0.2328312087689279
turtlebot_description 0.232500791713332
turtlebot_arm_moveit_demos 0.2307241410070523
turtlebot_arm_bringup 0.2294022741089882
turtlebot_rviz_launchers 0.2284678420795231
turtlebot_interactive_markers 0.2282293740531807
turtlebot3_panorama 0.2273231293707801
turtlebot 0.2272110815162024
turtlebot_arm_moveit_config 0.22705534438670777
turtlebot_actions 0.22670127612562896


In [142]:
scores["turtlebot_gazebo"] #0.16696136891889354
scores["rplidar_ros"]  #0.15153551060241577

0.15153551060241577

In [134]:
a = len(package_all.keys())
# package_all["turtlebot_gazebo"]
package_all["rt_usb_9axisimu_driver"]

{'robot': ['turtlebot4'],
 'sensor': ['rt corp. usb output 9-axis imu sensor module'],
 'function': [],
 'characteristics': ['-0.1148470789194107 -0.4843180179595947 0.16170334815979004 0.2290831059217453 0.3862266540527344 -0.4836290776729584 -0.19434218108654022 0.309590607881546 -0.10819730907678604 -0.5577605366706848 0.3077353537082672 0.026396101340651512 -0.37166503071784973 0.37197038531303406 0.08352403342723846 0.39752891659736633 -0.401959627866745 0.5252407193183899 -0.2970407009124756 0.22262926399707794 0.2127312421798706 -0.19883513450622559 -0.2613779306411743 -0.2150297462940216 -0.1289929449558258 0.0719621330499649 -0.04298270121216774 -0.2016775757074356 -0.8166974186897278 -0.17142939567565918 -0.40912100672721863 0.060875385999679565 0.17799831926822662 -0.09381190687417984 -0.15596066415309906 0.0921161025762558 -0.014398932456970215 -0.06661462038755417 0.05683565512299538 0.3798139989376068 -0.15118181705474854 -0.8758288621902466 0.4858991205692291 0.204901978

In [135]:
# fuzz.partial_ratio("rt", "turtlebot3")

In [157]:
ratio = fuzz.partial_ratio("turtlebot2", "turtlebot3")
print(ratio)
ratio = fuzz.partial_ratio("turtlebot2", "turtlebot2")
# fuzz.partial_ratio("turtlebot3", "turtlebot")
print(ratio)

90
100


In [146]:
from tqdm import tqdm
import time

test_d = {}
for i in range(60):
    test_d[i] = i
print(test_d)

for i in tqdm(test_d.items()):
    """
    代码
    """
    print(i)
    # 假设这代码部分需要0.05s，循环执行60次
    time.sleep(0.05)


{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59}


  3%|█▍                                        | 2/60 [00:00<00:03, 19.22it/s]

(0, 0)
(1, 1)
(2, 2)
(3, 3)


 10%|████▏                                     | 6/60 [00:00<00:02, 19.44it/s]

(4, 4)
(5, 5)
(6, 6)
(7, 7)


...

 90%|████████████████████████████████████▉    | 54/60 [00:02<00:00, 18.74it/s]

(52, 52)
(53, 53)
(54, 54)
(55, 55)


 97%|███████████████████████████████████████▋ | 58/60 [00:03<00:00, 18.56it/s]

(56, 56)
(57, 57)
(58, 58)
(59, 59)


100%|█████████████████████████████████████████| 60/60 [00:03<00:00, 18.71it/s]


In [58]:

# word1 = "I am an idiot"
# word2 = "michael is stupid"
# bc = BertClient(ip='localhost', check_version=False, check_length=False)
# vec1 = bc.encode([word1])
# # print(vec1)
# string = ' '.join(map(str, vec1.ravel().tolist()))
# # print(string)



# data_shape = vec1.shape
# print(data_shape)
# vec2 = np.fromstring(string, sep=' ').reshape((1,768))


# print(vec2)
# print(vec2.shape)


# # vec2 = np.ndarray(string)

# # type(vec2)


(1, 768)
[[-3.05952191e-01  6.15500510e-01 -5.02849638e-01 -6.28069341e-01
  -3.96398455e-01 -6.66799918e-02 -6.21556155e-02  6.04231179e-01
  -1.95092320e-01 -1.73862204e-01 -1.43248320e-01 -5.86824000e-01
  -1.34633183e-01  4.42531735e-01 -3.07732016e-01  3.07622224e-01
   1.36465445e-01  8.22881833e-02  5.12000173e-03  6.70854986e-01
  -1.08120359e-01  4.41043735e-01 -8.65872562e-01  9.35962796e-02
  -2.08369151e-01 -4.51294690e-01  5.69588356e-02 -2.45733067e-01
   6.49805963e-02  2.16230378e-01  3.89154941e-01  5.73705673e-01
   2.66966611e-01  4.65254992e-01 -7.75463954e-02 -3.24988216e-01
  -7.25363493e-02 -4.83782798e-01 -6.90067112e-01  1.97907925e-01
  -4.58944172e-01 -2.90900469e-01  1.01648815e-01  6.82343915e-02
  -7.19142780e-02 -3.53192657e-01 -1.08065911e-01 -1.70043483e-01
  -2.66843718e-02 -7.88914025e-01 -1.85458481e-01 -1.82695806e-01
  -8.84982049e-02  1.02178223e-01 -1.10074162e-01  5.51576257e-01
  -4.16893512e-01 -8.93596828e-01 -4.75774616e-01  4.18005019e-01
 

In [56]:
def match_pkg_by_file(file_name, file_type):
    pkg_list = []
    
    answer = g.run("MATCH (p:Package)- [rel:include] -> (l:%s {name:\"%s\"}) RETURN p.package_name"%(file_type, file_name)).data()
    for p in answer:
        pkg_list.append(p['p.package_name'])
#         print(len(answer))
    return pkg_list
            
    
    

In [57]:
launch = 'demo.launch'
pkgs = match_pkg_by_file(launch, 'Launch')
print(pkgs)

['mrpt_map', 'mrpt_localization', 'nfc_ros', 'fsrobo_r_moveit_config', 'svenzva_moveit', 'abb_irb2400_moveit_config', 'abb_irb6640_moveit_config', 'rqt_py_trees', 'prbt_moveit_config', 'capabilities', 'x_demo', 'motoman_mpl80_moveit_config', 'motoman_sia5d_moveit_config', 'motoman_csda10f_moveit_config', 'sciurus17_moveit_config', 'fetch_moveit_config', 'op3_demo', 'hironx_moveit_config', 'abb_irb1200_7_70_moveit_config', 'abb_irb1600_6_12_moveit_config', 'abb_irb120_moveit_config', 'abb_irb1200_5_90_moveit_config', 'abb_irb120t_moveit_config', 'nextage_moveit_config', 'staubli_rx160_moveit_config', 'seed_r7_samples', 'seed_r7_typeg2_arm_moveit_config', 'seed_r7_typef_moveit_config', 'seed_r7_typeg_moveit_config', 'seed_r7_typeg_arm_moveit_config', 'cob_moveit_bringup', 'panda_moveit_config', 'rviz_visual_tools', 'rb1_dual_ur3_moveit_config', 'roch_capabilities', 'rv7fl_moveit_config', 'rv4fl_moveit_config', 'niryo_one_moveit_config', 'moveit_visual_tools', 'motoman_ma2010_moveit_confi

In [58]:
action = 'Gripper.action'
pkgs = match_pkg_by_file(action, 'Action')
print(pkgs)

['svenzva_msgs', 'rail_manipulation_msgs']


In [59]:
# 'word1':98, 'word2':96, 'word3':92, ...
# probably, there are several pkgs for one device(word#n)
# region_type : robot/sensor
def match_pkg_by_region_word(region_word, region_type):
    pkg_list = []
    
    answer = g.run("MATCH (p:Package)- [rel:apply_to] -> (l:%s {name:\"%s\"}) RETURN p.package_name"%(region_type, region_word)).data()
    for p in answer:
        pkg_list.append(p['p.package_name'])
#         print(len(answer))
    return pkg_list
            

In [60]:
sensor = 'rospeex'
pkgs = match_pkg_by_region_word(sensor, 'Sensor')
print(pkgs)

['rospeex_audiomonitor', 'rospeex_core', 'rospeex_if', 'rospeex_launch', 'rospeex_msgs', 'rospeex_samples', 'rospeex_webaudiomonitor', 'rospeex']


In [66]:
#given a sentence as input, now we need to extract region words in it.
import pandas as pd
import numpy as np

# remove "up" and "down"
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn']
for w in ['!',',','.','?','','s','\n','\t', '(', ')']:
    stopwords.append(w)

def remove_func(phras):
    res = phras.strip(',')
    res = phras.strip('.')
#     res = res.strip('(')
#     res = res.strip(')')
    return res


def extract_keywords(raw_sentence):
    # deal with the text
    sentence = []
    raw_sentence = raw_sentence.strip('-')
    raw_sentence = raw_sentence.strip('.')
    tokens = raw_sentence.split()
    phrase_start = None
    for word in tokens:
        word = remove_func(word)
        if len(word) != 1 and (word.lower() not in stopwords):
            if phrase_start is None:
                phrase_start = word
            else:
                phrase_start += f' {word}'
        elif phrase_start:

            sentence.append(phrase_start)
            phrase_start = None
        else:
            continue
    if phrase_start:
        sentence.append(phrase_start)

#     keywords =  ','.join(sentence)
    return sentence



In [67]:
# test: extract keywords from a given query sentence from user
sentence = "use turtlebot2 with realsense camera to do mapping by gmapping algorithm"

keywords = extract_keywords(sentence)

print(keywords)


['use turtlebot2', 'realsense camera', 'mapping', 'gmapping algorithm']


In [72]:
# use fuzzy match to region words
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def extract_region_word(keywords):
    # identify robot entity
    cur_dir = '/Users/wshuo/PycharmProjects/MyKnowledgeGraph'
    robot_path = os.path.join(cur_dir, 'data/robots.txt')
    robot_wds = [i.strip().lower() for i in open(robot_path)]
    robot_wds = list(set(robot_wds))
    
    for word in keywords:
        fuzzy_match_results = process.extract(word, robot_wds, limit=2)
#         match_region = fuzzy_match_results[0]
        print(fuzzy_match_results)
    pass

    sensor_path = os.path.join(cur_dir, 'data/sensors.txt')
    sensor_wds = [i.strip().lower() for i in open(sensor_path)]
    sensor_wds = list(set(sensor_wds))
    
    for word in keywords:
        fuzzy_match_results = process.extract(word, sensor_wds, limit=2)
#         match_region = fuzzy_match_results[0]
        print(fuzzy_match_results)
    pass



extract_region_word(keywords)
        
# remove "bo" from the dictionary of robot and sensor


# [('turtlebot2', 95), ('bo', 90)]
# [('reem-c', 60), ('caster', 60)]
# [('thormang', 53), ('astec pelican and hummingbird', 51)]
# [('ari', 60), ('quori', 54)]






[('turtlebot2', 95), ('bo', 90)]
[('reem-c', 60), ('caster', 60)]
[('thormang', 53), ('astec pelican and hummingbird', 51)]
[('ari', 60), ('quori', 54)]
[('motion analysis motion capture system', 49), ('chip robotics imu sensor (bno080)', 49)]
[('roboception rc_visard stereo camera (onboard imu, pose estimation and slam)', 86), ('zed stereo camera tracking', 86)]
[('mesa imaging swissranger devices (3000/4000/4500)', 64), ('cmucam_png', 59)]
[('vrpn', 45), ('pmad', 45)]


In [73]:
word1 = 'realsense camera'
word2 = 'Intel® RealSense™ camera'

score = fuzz.partial_ratio(word1, word2)
print(score)

88


In [79]:
robot = 'TurtleBot2'
pkgs = match_pkg_by_region_word(robot, 'Robot')
print(pkgs)

[]


In [76]:
robot = 'realsense camera'
pkgs = match_pkg_by_region_word(robot, 'Sensor')
print(pkgs)

[]


In [81]:
robot = 'turtlebot3'
pkgs = match_pkg_by_region_word(robot, 'Robot')
print(pkgs)

['turtlebot3_autorace_construction_camera', 'turtlebot3_msgs', 'turtlebot_create_desktop', 'turtlebot_dashboard', 'turtlebot_interactions', 'turtlebot_interactive_markers', 'turtlebot_rviz_launchers', 'turtlebot_msgs', 'turtlebot3', 'turtlebot3_bringup', 'turtlebot3_description', 'turtlebot3_example', 'turtlebot3_navigation', 'turtlebot3_slam', 'turtlebot3_teleop', 'turtlebot_arm', 'turtlebot_arm_block_manipulation', 'turtlebot_arm_bringup', 'turtlebot_arm_description', 'turtlebot_arm_ikfast_plugin', 'turtlebot_arm_kinect_calibration', 'turtlebot_arm_moveit_config', 'turtlebot_arm_moveit_demos', 'turtlebot_arm_object_manipulation', 'turtle_actionlib', 'turtle_tf', 'turtle_tf2', 'turtlebot_actions', 'turtlebot_apps', 'turtlebot_calibration', 'turtlebot_create', 'turtlebot_follower', 'turtlebot_navigation', 'turtlebot_rapps', 'turtlebot_gazebo', 'turtlebot_simulator', 'turtlebot_stage', 'turtlebot_stdr', 'turtlebot3_applications_msgs', 'turtlebot3_autorace', 'turtlebot3_autorace_camera',

In [82]:
keywords = ['rplidar']
extract_region_word(keywords)

[('ari', 72), ('ar10', 60)]
[('rplidar a1/2/3/s1/s2 laser(c++)', 90), ('rplidar 360 laser scanner driver(python)', 90)]


In [84]:
robot = 'rplidar 360 laser scanner driver(python)'
pkgs = match_pkg_by_region_word(robot, 'Sensor')
print(pkgs)

['rplidar_ros']
